# Getting assembled data
Try to download complete assembled genomes Klebsiella Pneumoniae from NCBI genome

# REFERENCE DATA

## 1) Get genomes with NCBI eutils : FAIL
Tried to get the genomes with NCBI eutils but this failed because the database genome is not supported by eutils


IN TERMINAL:

#install eutils
python3 --version

#first get miniconda:
#download from https://conda.io/miniconda.html  
cd ~/Downloads/
bash Miniconda3-latest...
#add channels
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge


pip3 install eutils
#does not work
conda install -c bioconda eutils #looking for package eutils in channel bioconda
#no packages found

sudo pip3 install eutils
#worked :)

In [2]:
#!/usr/bin/python
from eutils import Client

#set API
eclient = Client(api_key="bccfde389fa10feecea94b5b64ef7028f309")
#search genomes
genome_esearch = eclient.esearch(db='genome',term='Klebsiella pneumoniae')

In [13]:
print("\n\nResults of gene esearch:\n{}".format(genome_esearch))
obj_summary_list = dir(genome_esearch)
print(obj_summary_list)

print("total list=",genome_esearch.ids)
#genome 815



Results of gene esearch:
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_root_tag', '_xml_root', 'count', 'ids', 'retmax', 'retstart', 'webenv']
total list= [24383, 815]


In [18]:
genome_efetch = eclient.efetch(db='genome', id=genome_esearch.ids[1])
print(genome_efetch)
#database genome is not supported by eutils -> search new method

EutilsError: database genome is not currently supported by eutils



## 2) Ncbi-genome-download: fail
Used already developed tool to download all complete genomes of a species on NCBI, see github: https://github.com/kblin/ncbi-genome-download  



In [ ]:
#in terminal: successfully installed
sudo pip3 install ncbi-genome-download

In [ ]:
#in terminal
ncbi-genome-download --assembly-level complete --format fasta --genus "Klebsiella pneumoniae" bacteria
#1172 complete genomen

Resultaat 1172 complete genomen: fasta files (fna.gz)
import os
os.listdir("/home/hannelore/PROJECTHH/dataKP-assembly/bacteria")

For unknown reason only 390 genomes downloaded, enough for the trials

REMARK: zijn MD5SUMS files en nog niet de echte fasta's...

In [ ]:
#een specifiek tax ID:
ncbi-genome-download --taxid 511145 bacteria

In [ ]:
#RETRY/SUMMARY
#download via ncbi-genome-download: see get assembled data
sudo pip3 install ncbi-genome-download #install
ncbi-genome-download --taxid 573 bacteria
#remark downloads ALL refs: adjust:
ncbi-genome-download --assembly-level complete bacteria --taxid 573 --format fasta
#gives format in MD5SUM (can not use this, search fornew tool)

## 3) in the browser : OK
works but not good for automatic script

download all assemblies on NCBI > Assembly
* go to https://www.ncbi.nlm.nih.gov/assembly/
* type txid ... in search box
* indicate "complete genome" at assembly level
* "download assemblies" (fasta)

Entrez query: 573[txid] AND "complete genome"[filter] AND "latest refseq"[filter]

## 4) EDirect

#### E-utilities 
is an API that can be used to access PubMed and other NCBI data.
#### EDirect 
is a set of tools that allow you to more easily use the E-utilities API in a Unix environment. (for command line!)

In [ ]:
#install eutils-1
pip3 install --user eutils
#installation works but can not import in python script :s

#install eutils-2
pip install eutils
#does work, also with python3 :s


In [ ]:
#!/usr/bin/python3

from eutils import Client

eclient = Client(api_key="f7bba41c57271397bbc91985bce382bf6***")
print("\nUsing NCBI E-utilities in Python\n")

#does work
gene_esearch = eclient.esearch(db='gene',term='TNF')
#does not work
assembly_esearch = eclient.esearch(db='assembly',term='klebsiella pneumoniae')
#does not work
assembly_esearch = eclient.esearch(db='assembly',term='573[txid] AND "complete genome"[filter] AND "latest refseq"[filter]')
#does not work
esearch -db assembly -query '573[txid] AND "complete genome"[filter] AND "latest refseq"[filter]'

In [ ]:
%%bash
#install EDirect
#https://astrobiomike.github.io/unix/ncbi_eutils
conda install -y -c conda-forge -c bioconda -c defaults entrez-direct

#getting the accession numbers:
cd /home/hannelore/PROJECTHH/Data/dataKP-assembly/edirect
esearch -db assembly -query '573[txid] AND "complete genome"[filter] AND "latest refseq"[filter]' \
| esummary | xtract -pattern DocumentSummary -element AssemblyAccession > accessions573.txt

#get 1 assembly (RefSeq)
esearch -db assembly -query GCF_006538345.1 | elink -target nucleotide -name \
        assembly_nuccore_refseq | efetch -format fasta > GCF_006538345.1.fa
        
#get 1 assembly (genbank)
esearch -db assembly -query GCA_006538345.1 | elink -target nucleotide -name \
        assembly_nuccore_insdc | efetch -format fasta > GCA_006538345.1.fa
        
#alternative to fetch them all:
#Bioinf Tools: bit-dl-ncbi-assemblies
#install
conda install -c conda-forge -c bioconda -c astrobiomike bit python=3.7
#get all data from file
bit-dl-ncbi-assemblies -w accessions573.txt -f fasta -j 10
#-j = number of donwloads in parallel
#takes +/- 15min because a lot of references, if used in script maybe store some in database

# TEST DATA